# Imports

In [ ]:
import os
import sys
import time
import math
import gzip
import json
from pathlib import Path
import csv
import re
import numpy as np
import asyncpg
import glob
import gzip
import pandas as pd
def prettyprint(item):
    print(json.dumps(item, indent=4, sort_keys=False))


In [ ]:

pgsql_params = dict(
    dsn=os.getenv("PGSQL_URL"),
    server_settings={'search_path': "edsm"}
)
pgpool = await asyncpg.create_pool(**pgsql_params)

In [ ]:
print(await pgpool.fetch("SELECT * FROM edsm.systems WHERE name = $1", "Ix"))

# Setting up the big cubes

In [ ]:
data = []
count = 0

for cy in range(-3000, 3000, 200): # n=30
    count += 1
    qr = await pgpool.copy_records_to_table("volumes", records=[(cx,cy,cz)  for cx in range(-30000, 30000, 200) for cz in range(-24000, 66000,200)])
    print(f"{count}/{30}\t{round(100*count/30,2)}%")



In [ ]:
# Add column with distance to Sol
await pgpool.execute("""
    ALTER TABLE edsm.volumes ADD COLUMN distance integer; 
""")

In [ ]:
await pgpool.execute("""
    UPDATE  edsm.volumes 
    SET distance = |/((c200x+100)^2 + (c200y+100)^2 + (c200z+100)^2); 
""")

In [ ]:
assert False, "Only run after recreating The Voids"
qr = await pgpool.fetch("""
    SELECT v.*
    FROM edsm.volumes v
    LEFT JOIN edsm.systems s
    ON v.c200x = s.c200x AND v.c200y = s.c200y AND v.c200z = s.c200z 
    WHERE s.name IS NULL
    """)

await pgpool.copy_records_to_table("voids", records=qr)

In [ ]:
# Add column with distance to Sol
await pgpool.execute("""
    ALTER TABLE edsm.voids ADD COLUMN distance integer; 
""")

In [ ]:
await pgpool.execute("""
    UPDATE  edsm.voids 
    SET distance = |/((c200x+100)^2 + (c200y+100)^2 + (c200z+100)^2); 
""")

# scratchpad


In [ ]:
import sys
import time
from pathlib import Path
import gzip
import json
# systemsWithCoordinates7days
def edc_dbfilereader(filename, verbose=False):
    """
        Opens 'filename' as generator for eddb style objects
    """

    filesize=Path(filename).stat().st_size
    chunksize = 64 * 1024 * 1024
    est_count = int(6.7*filesize/chunksize) + 1
    print(f"Reading {filename}, {round(filesize/(1024*1024),1)} Mb in approx {est_count} chunks")

    count = 0
    system_count = 0
    item = None

    start = time.process_time()
    #try:

    with gzip.open(filename, 'rt') as jsonfile:

        while True:
            count += 1
            chunk = jsonfile.readlines(chunksize)
            if chunk:
                for line in chunk:
                    if len(line) < 5:
                        continue

                    yield json.loads(line.rstrip(',\n\r '))

                    system_count += 1

                yield {}
                sys.stdout.write(f"\r{count}/{est_count}\t{100*count/est_count:3.2f}%, {int(system_count / (time.process_time() - start)):6} /s, {system_count:9} systems, {((est_count - count) * (time.process_time() - start)/count):5.1f} seconds remaining")

            else:
                print(f"\nEmpty chunk -> Done! Imported {system_count} systems in {round(time.process_time() - start,1)} seconds")
                break

    tpl = (time.process_time() - start)/system_count
    sys.stdout.write(f"\n{ (time.process_time() - start)} seconds {system_count} systems, per system {round(1000000*tpl,2)} us")


In [ ]:
print(f"Adding indexes ...")
await pgpool.execute(f"""
    CREATE INDEX IF NOT EXISTS systems_x_idx ON edsm.systems (x);
    CREATE INDEX IF NOT EXISTS systems_y_idx ON edsm.systems (y);
    CREATE INDEX IF NOT EXISTS systems_z_idx ON edsm.systems (z); 
    CREATE INDEX IF NOT EXISTS systems_name_idx ON edsm.systems (name);
""")

In [ ]:


await pgpool.execute("""
    ALTER TABLE edsm.systems ADD COLUMN N boolean default false; 
""")

In [ ]:
#filename = "data/galaxy_7days.json.gz"
filename=os.path.join('f:', os.sep, 'data', 'eddb', 'galaxy_1month-2023-01-03.json'+'.gz')

In [ ]:
data = []
updated = 0

for item in edc_dbfilereader(filename):
    if not item:
        #qr = await pgpool.copy_records_to_table("systems", records=data)
        qr = await pgpool.executemany(
            """INSERT INTO edsm.systems (name, x, y, z, n) 
                VALUES ($1, $2, $3, $4, $5) 
                ON CONFLICT (name) DO UPDATE SET n=$5, x=$2, y=$3, z=$4
            """, data)
        updated += len(data)
        data = []
        time.sleep(4)
        continue

    bodies = item.get('bodies',[{}])

    neutron = [bool(B.get('subType','')=='Neutron Star') for B in bodies if B.get('mainStar')]        
    coords = item.get('coords')
    #coordinates = [coords[k] for k in ['x','y','z']]
    data.append(
        [
            item.get('name'), 
            *[coords[k] for k in ['x','y','z']],
            bool(bool(neutron) and all(neutron))
        ]
        #[item.get('name')] + [coords[k] for k in ['x','y','z']] + [len(neutron)>0 and all(neutron)]
    )
print(f"\nUpdated {updated} systems with neutron star primary")

In [ ]:
#filename = "data/galaxy_7days.json.gz"
filename=os.path.join('f:', os.sep, 'data', 'eddb', 'galaxy.json'+'.gz')

In [ ]:
data = []
updated = 0

for item in edc_dbfilereader(filename):
    if not item:
        #qr = await pgpool.copy_records_to_table("systems", records=data)
        qr = await pgpool.executemany(
            """INSERT INTO edsm.systems (name, x, y, z, n) 
                VALUES ($1, $2, $3, $4,TRUE) 
                ON CONFLICT (name) DO UPDATE SET n = TRUE
            """, data)
        updated += len(data)
        data = []
        time.sleep(2)
        continue

    bodies = item.get('bodies',[{}])
    if bodies:
    
        #mainstar = [B.get('subType','-')[0] for B in bodies if B.get('mainStar')]
        neutron = [bool(B.get('subType','')=='Neutron Star') for B in bodies if B.get('mainStar')]
        if bool(neutron) and all(neutron):
            coords = item.get('coords')
            #coordinates = [coords[k] for k in ['x','y','z']]
            data.append(
                [
                    item.get('name'), 
                    *[coords[k] for k in ['x','y','z']]
                ]
                #[item.get('name')] + [coords[k] for k in ['x','y','z']] + [len(neutron)>0 and all(neutron)]
            )

print(f"\nUpdated {updated} systems with neutron star primary")

In [ ]:
82/1000000 * 70000000

# Reading EDSM json



## Systems

In [ ]:
filename = "data/systemsWithCoordinates.json.gz"

# 2022-04-17: 552.40625 seconds 70750837 systems, per system 7.81 us, est: 404.86962424380926
# 2022-04-10: 882.5625 seconds 70583001 systems, per system 12.5 us, est: 648.3858730978299
#  with 900 duplicate systems
#assert False, "Completed, don't do this again unless you've dropped and recreated the table"

# Drop te existing table:
await pgpool.execute(f"DROP TABLE edsm.systems;")
await pgpool.execute(f"""
    CREATE TABLE IF NOT EXISTS edsm.systems (
        name text NOT NULL,
        x DOUBLE PRECISION,
        y DOUBLE PRECISION,
        z DOUBLE PRECISION
    );
""")


filesize=Path(filename).stat().st_size
chunksize = 64 * 1024 * 1024
est_count = int(8*filesize/chunksize) + 1
print(f"Reading {filename}, {round(filesize/(1024*1024),1)} Mb in approx {est_count} chunks")

count = 0
system_count = 0
columns = slice(2,6)
start = time.process_time()
with gzip.open(filename, 'rt') as jsonfile:

    firstline = jsonfile.readline()

    while True:
        count += 1
        chunk = jsonfile.readlines(chunksize)
        if chunk:
            data = []
            for line in chunk:
                if len(line) < 5:
                    continue
                item = json.loads(line[0:-2]) if line[-2] == "," else json.loads(line)
                coords = item.get('coords')
                coordinates = [coords[k] for k in coords]
                data.append(
                    [item.get("name")] + coordinates #+
                    #[int(20*math.floor(v/20)) for v in coordinates] +
                    #[int(200*math.floor(v/200)) for v in coordinates]
                )
                
            qr = await pgpool.copy_records_to_table("systems", records=data)
            system_count += len(data)
            sys.stdout.write(f"{count}/{est_count}\t{100*count/est_count:3.2f}%, {int(system_count / (time.process_time() - start)):6} /s, {system_count:9} systems, {((est_count - count) * (time.process_time() - start)/count):5.1f} seconds remaining\r")
            #print(f"{count}/{est_count}\t{round(100*count/est_count,2)}%,\t{int(system_count / (time.process_time() - start))} /s,\t{system_count} systems, \t{round(((est_count - count) * (time.process_time() - start)/count),1)} seconds remaining")
            
        else:    
            print(f"\nEmpty chunk -> Done! Imported {system_count} systems in {round(time.process_time() - start,1)} seconds")
            break

            

tpl = (time.process_time() - start)/system_count
print(f"{ (time.process_time() - start)} seconds {system_count} systems, per system {round(1000000*tpl,2)} us") 

In [ ]:
pd.DataFrame(data)

In [ ]:
print(f"Adding indexes ...")
await pgpool.execute(f"""
    CREATE INDEX IF NOT EXISTS systems_x_idx ON edsm.systems (x);
    CREATE INDEX IF NOT EXISTS systems_y_idx ON edsm.systems (y);
    CREATE INDEX IF NOT EXISTS systems_z_idx ON edsm.systems (z); 
    CREATE INDEX IF NOT EXISTS systems_name_idx ON edsm.systems (name);
""")

In [ ]:
print("Removing duplicates by system name")
await pgpool.execute("""
    DELETE FROM edsm.systems a
    WHERE   a.ctid <> (SELECT min(b.ctid)
                     FROM   edsm.systems b
                     WHERE  a.name = b.name );"""
)

In [ ]:
print(f"Adding unique index on system name ...")
await pgpool.execute(f"""
    DROP INDEX systems_name_idx ;
    CREATE UNIQUE INDEX IF NOT EXISTS systems_name_unique ON edsm.systems (name)
""")

### Update

In [ ]:
glob.glob(os.path.join('data', "systemsWithCoordinates7days-*.json.gz"))

In [ ]:

start = time.process_time()

for filename in sorted(glob.glob(os.path.join('data', "systemsWithCoordinates7days-*.json.gz"))):
    
    filesize=Path(filename).stat().st_size
    chunksize = 4 * 1024 * 1024
    est_count = int(8*filesize/chunksize) + 1
    print(f"Reading {filename}, {round(filesize/(1024*1024),1)} Mb in approx {est_count} chunks")

    count = 0
    system_count = 0
    columns = slice(2,6)
    data=[]

    with gzip.open(filename, 'rt') as jsonfile:

        firstline = jsonfile.readline()

        while True:
            count += 1
            chunk = jsonfile.readlines(chunksize)
            if chunk:
                data = []
                for line in chunk:
                    if len(line) < 5:
                        continue
                    item = json.loads(line[0:-2]) if line[-2] == "," else json.loads(line)

                    coords = item.get('coords')
                    coordinates = [coords[k] for k in coords]
                    data.append(
                        [item.get("name")] + coordinates #+
                        #[int(20*math.floor(v/20)) for v in coordinates] +
                        #[int(200*math.floor(v/200)) for v in coordinates]
                    )

                qr = await pgpool.executemany(
                    """INSERT INTO edsm.systems (name, x, y, z) 
                        VALUES ($1, $2, $3, $4) 
                        ON CONFLICT DO NOTHING
                    """, data)
                system_count += len(data)
                sys.stdout.write(f"{count}/{est_count}\t{100*count/est_count:3.2f}%, {int(system_count / (time.process_time() - start)):6} /s, {system_count:9} systems, {((est_count - count) * (time.process_time() - start)/count):5.1f} seconds remaining\r")
                #print(f"{count}/{est_count}\t{round(100*count/est_count,2)}%,\t{int(system_count / (time.process_time() - start))} /s,\t{system_count} systems, \t{round(((est_count - count) * (time.process_time() - start)/count),1)} seconds remaining")
                continue

            print(f"\nEmpty chunk -> Done! Imported {system_count} systems in {round(time.process_time() - start,1)} seconds")
            break



tpl = (time.process_time() - start)/system_count
print(f"{ (time.process_time() - start)} seconds {system_count} systems, per system {round(1000000*tpl,2)} us, est: {tpl*51854708}") 

### Populated systems 

In [ ]:
filename = "data/systemsPopulated.json.gz"

filesize=Path(filename).stat().st_size
chunksize = 64 * 1024 * 1024
est_count = int(8*filesize/chunksize) + 1
print(f"Reading {filename}, {round(filesize/(1024*1024),1)} Mb in approx {est_count} chunks")

count = 0
system_count = 0
columns = slice(2,6)
start = time.process_time()
with gzip.open(filename, "rt") as jsonfile:
    firstline = jsonfile.readline()

    while True:
        count += 1
        chunk = jsonfile.readlines(chunksize)
        if chunk:
            data = []
            for line in chunk:
                if len(line) < 5:
                    continue
                item = json.loads(line[0:-2]) if line[-2] == "," else json.loads(line)
                data.append(
                    [item.get("name"), float(item.get("population") if item.get("population") else 0.0), item.get('security')]
                )

            qr = await pgpool.executemany(
                """INSERT INTO edsm.populated (systemname, population, security) 
                    VALUES ($1, $2, $3) 
                    ON CONFLICT (systemname) DO UPDATE SET 
                        population = $2,
                        security = $3
                """, data)
            #qr = await pgpool.copy_records_to_table("populated", records=data)
            system_count += len(data)
            print(f"{count}/{est_count}\t{round(100*count/est_count,2)}%,\t{int(system_count / (time.process_time() - start))} /s,\t{system_count} systems, \t{round(((est_count - count) * (time.process_time() - start)/count),1)} seconds remaining")
            continue
            
        print(f"Empty chunk -> Done! Imported {system_count} systems in {round(time.process_time() - start,1)} seconds")
        break

            

tpl = (time.process_time() - start)/system_count
print(f"{ (time.process_time() - start)} seconds {system_count} systems, per system {round(1000000*tpl,2)} us, est: {tpl*51854708}") 

In [ ]:
item

### Powerplay

In [ ]:
filename = "data/powerPlay.json.gz"

filesize=Path(filename).stat().st_size
chunksize = 64 * 1024 * 1024
est_count = int(8*filesize/chunksize) + 1
print(f"Reading {filename}, {round(filesize/(1024*1024),1)} Mb in approx {est_count} chunks")

count = 0
system_count = 0
columns = slice(2,6)
start = time.process_time()
with gzip.open(filename, 'rt') as jsonfile:
    firstline = jsonfile.readline()

    while True:
        count += 1
        chunk = jsonfile.readlines(chunksize)
        if chunk:
            data = []
            for line in chunk:
                if len(line) < 5:
                    continue
                item = json.loads(line[0:-2]) if line[-2] == "," else json.loads(line)
                data.append(
                    [item.get(k) for k in ["power", "name","powerState","allegiance"]] 
                )
                
            qr = await pgpool.executemany(
                """INSERT INTO edsm.powers (power, systemname, powerstate, allegiance) 
                    VALUES ($1, $2, $3, $4) 
                    ON CONFLICT (power, systemname) DO UPDATE SET 
                        powerstate = $3,
                        allegiance = $4
                """, data)
            system_count += len(data)
            print(f"{count}/{est_count}\t{round(100*count/est_count,2)}%,\t{int(system_count / (time.process_time() - start))} /s,\t{system_count} systems, \t{round(((est_count - count) * (time.process_time() - start)/count),1)} seconds remaining")
            continue
            
        print(f"Empy chunk -> Done! Imported {system_count} systems in {round(time.process_time() - start,1)} seconds")
        break

            

tpl = (time.process_time() - start)/system_count
print(f"{ (time.process_time() - start)} seconds {system_count} systems, per system {round(1000000*tpl,2)} us, est: {tpl*51854708}") 

In [ ]:
item

## Get data from logs

In [ ]:
logpath = "/Users/fenke/Saved Games/Frontier Developments/Elite Dangerous"
logfiles = glob.glob(os.path.join(logpath, "journal.*"))

last_system = {
    "timestamp": "2000-01-01",
    "StarSystem": None
}
data = {}
events = {}
jumps = {}
scans = {}
bodies = {}
systems = {}

scan_events = set([
        "SAAScanComplete",
        "SAASignalsFound",
        "Scan",
        "FSSDiscoveryScan",
        #"FSSSignalDiscovered",
        "FSSAllBodiesFound",
        "FSDJump"
    ])
jump_events = set([
    "StartJump",
    "FSDJump",
    "FuelScoop"
])
excluded_events = set([
    "ReceiveText",
    "Location",
    "Commander",
    "Materials",
    "Rank",
    "Progress",
    "Reputation",
    "LoadGame",
    "EngineerProgress",
    "Music",
    "Missions",
    "Loadout",
    "Music",
    "Statistics",
    "Cargo",
    "SupercruiseEntry"
])

async with pgpool.acquire() as pgconnection: 
    prepped_query = await pgconnection.prepare(
        """INSERT INTO edsm.systems (name, x, y, z) 
            VALUES ($1, $2, $3, $4) 
            ON CONFLICT DO NOTHING
        """
    )

    columns = slice(2,6)
    start = time.process_time()
    system_count = 0
    for filename in logfiles[-9:]:

        filesize=Path(filename).stat().st_size
        chunksize = 1 * 1024 * 1024
        est_count = int(filesize/chunksize) + 1
        #print(f"Reading {filename}, {round(filesize/(1024*1024),1)} Mb in approx {est_count} chunks")
        count = 0

        with open(filename, "rt") as jsonfile:
            firstline = jsonfile.readline()

            while True:
                count += 1
                chunk = jsonfile.readlines(chunksize)
                if chunk:
                    #data = []
                    for line in chunk:
                        if len(line) < 5:
                            continue
                        item = json.loads(line[0:-2]) if line[-2] == "," else json.loads(line)
                        if item.get("event") in excluded_events:
                            continue
                            
                        timestamp = item.get("timestamp")
                        eventname = item.get("event")
                        system_name = item.get("SystemName", item.get("StarSystem"))
                        if system_name and system_name not in systems:
                            systems[system_name] = dict(bodies=dict())
                            system = systems[system_name]
                        
                        if eventname in jump_events and item.get("JumpType") != "Supercruise":
                            if timestamp not in jumps:
                                jumps[timestamp] = []
                            jump = jumps.get(timestamp)
                            jump.append(item)
                            
                        elif eventname in scan_events:
                            body_name = item.get("BodyName")
                            if body_name not in bodies:
                                bodies[body_name] = {}
                            body = bodies[body_name]
                            
                            if system_name and body_name and body_name not in system["bodies"]:
                                system["bodies"][body_name] = body
                                
                            
                            for key in ["StarSystem", "DistanceFromArrivalLS","ProbesUsed", "WasDiscovered", "WasMapped", "Landable"]: # items
                                body[key] = item.get(key, body.get(key, None))
                            
                            #for key in ["Signals", "Materials"]: # lists
                            
                            if timestamp not in scans:
                                scans[timestamp] = []
                            scan = scans.get(timestamp)
                            scan.append(item)
                            
                        else:
                            if timestamp not in events:
                                events[timestamp] = []
                            event = events.get(timestamp)
                            event.append(item)
                        
                        if item.get("event") == "FSDJump":
                            coordinates = item.get('StarPos')
                            data[item.get("StarSystem")] = coordinates + [int(20*math.floor(v/20)) for v in coordinates] +  [int(200*math.floor(v/200)) for v in coordinates]
                            if timestamp > last_system.get("timestamp"):
                                last_system = {k:item[k] for k in last_system }
                            
                        #all_events.append(item.get("event"))
                        #data.append( [item.get(k) for k in ["name","systemName", "type","distanceToArrival"]]  )
                    continue # -> while


                #print(f"Empy chunk -> Done! Imported {system_count} systems in {round(time.process_time() - start,1)} seconds")
                break
                
        #await prepped_query.executemany([[S] + data[S] for S in data])
        system_count += len(data)
        data = {}
        
    #print(f"{count}/{est_count}\t{system_count}\tsystems,\t{int(system_count / (time.process_time() - start))} /s,\t{round(100*count/est_count,2)}%, {round(((est_count - count) * (time.process_time() - start)/count),1)} remaining")


    if system_count > 0:
        tpl = (time.process_time() - start)/system_count
        print(f"{ (time.process_time() - start)} seconds {system_count} systems, per system {round(1000000*tpl,2)} us, est: {tpl*51854708}") 

In [ ]:
{T:jumps[T] for T in jumps if jumps[T][0]['event']=='FSDJump'}

In [ ]:
jumps

In [ ]:
prettyprint(systems)

In [ ]:
prettyprint(scans)

In [ ]:
prettyprint(last_system)
prettyprint([R.get("name") for R in await find_nearby_systems(last_system.get("StarSystem"), 10) ])

## Removing Duplicates

### One query to remove them all

In [ ]:
await pgpool.execute("""
    DELETE FROM edsm.systems a
    WHERE   a.ctid <> (SELECT min(b.ctid)
                     FROM   edsm.systems b
                     WHERE  a.name = b.name );"""
)

### Another ONE bites the dust

In [ ]:
await pgpool.execute("""
    DELETE FROM edsm.systems a USING (
          SELECT MIN(ctid) as ctid, name
            FROM edsm.systems 
            GROUP BY name HAVING COUNT(*) > 1
          ) b
          WHERE a.name = b.name 
          AND a.ctid <> b.ctid
    """)

In [ ]:
qr = await pgpool.fetch("SELECT name, count(1) FROM edsm.systems GROUP BY name HAVING count(1) > 1")

async with pgpool.acquire() as pgconnection: 
    prepped_query = await pgconnection.prepare(
        """
            DELETE FROM edsm.systems a
            WHERE   a.name = $1 AND
                    a.ctid <> (SELECT min(b.ctid)
                             FROM   edsm.systems b
                             WHERE  a.name = b.name );        
        """
    )
    await prepped_query.executemany([[R.get("name")] for R in qr])

In [ ]:
[R.get("name") for R in qr]

In [ ]:
qr

## Stations

In [ ]:
filename = "data/stations.json"
filesize=Path(filename).stat().st_size
chunksize = 64 * 1024 * 1024
est_count = int(filesize/chunksize) + 1
print(f"Reading {filename}, {round(filesize/(1024*1024),1)} Mb in approx {est_count} chunks")

count = 0
system_count = 0
columns = slice(2,6)
start = time.process_time()
with open(filename, "rt") as jsonfile:
    firstline = jsonfile.readline()

    while True:
        
        count += 1
        chunk = jsonfile.readlines(chunksize)
        if chunk:
            data = []
            for line in chunk:
                if len(line) < 5:
                    continue
                item = json.loads(line[0:-2]) if line[-2] == "," else json.loads(line)
                data.append( [item.get(k) for k in ["name","systemName", "type","distanceToArrival"]]  )
                
            qr = await pgpool.copy_records_to_table("stations", records=data)
            system_count += len(data)
            print(f"{count}/{est_count}\t{system_count}\tsystems,\t{int(system_count / (time.process_time() - start))} /s,\t{round(100*count/est_count,2)}%, {round(((est_count - count) * (time.process_time() - start)/count),1)} remaining")
            continue
            
        print(f"Empy chunk -> Done! Imported {system_count} systems in {round(time.process_time() - start,1)} seconds")
        break

            
if system_count > 0:
    tpl = (time.process_time() - start)/system_count
    print(f"{ (time.process_time() - start)} seconds {system_count} systems, per system {round(1000000*tpl,2)} us, est: {tpl*51854708}") 

In [ ]:
await pgpool.execute(
        """
            DELETE FROM edsm.stations a
            WHERE   a.ctid <> (SELECT min(b.ctid)
                             FROM   edsm.stations b
                             WHERE  a.name = b.name AND a.systemname = b.systemname);        
        """
    )

# Reading EDDN

In [ ]:
#filename = "data/galaxy_1day.json.gz"
filename = "E:/data/eddb/galaxy_1day.json.gz"

filesize=8*Path(filename).stat().st_size
chunksize = 64 * 1024 * 1024
est_count = int(filesize/chunksize) + 1
print(f"Reading {filename}, estimated {round(filesize/(1024*1024),1)} Mb in approx {est_count} chunks")

In [ ]:
count = 0
system_count = 0
columns = slice(2,6)
start = time.process_time()
with gzip.GzipFile(filename, 'r') as jsonfile:
    firstline = jsonfile.readline()
    for t in range(0,4):
        line = jsonfile.readline().decode('utf-8')
        #print(line, "\n")
        prettyprint(json.loads(line[0:-2]) if line[-2] == "," else json.loads(line))
    

    while True:
        break
        count += 1
        chunk = jsonfile.readlines(chunksize)
        if chunk:
            data = []
            for line in chunk:
                if len(line) < 5:
                    continue
                item = json.loads(line[0:-2]) if line[-2] == "," else json.loads(line)
                coords = item.get('coords')
                coordinates = [coords[k] for k in coords]
                data.append(
                    [item.get("name")] + 
                    coordinates +
                    [int(20*math.floor(v/20)) for v in coordinates] +
                    [int(200*math.floor(v/200)) for v in coordinates]
                )
                
            qr = await pgpool.copy_records_to_table("stations", records=data)
            system_count += len(data)
            print(f"{count}/{est_count}\t{system_count}\tsystems,\t{int(system_count / (time.process_time() - start))} /s,\t{round(100*count/est_count,2)}%, {round(((est_count - count) * (time.process_time() - start)/count),1)} remaining")
            continue
            
        print(f"Empy chunk -> Done! Imported {system_count} systems in {round(time.process_time() - start,1)} seconds")
        break

            
if system_count > 0:
    tpl = (time.process_time() - start)/system_count
    print(f"{ (time.process_time() - start)} seconds {system_count} systems, per system {round(1000000*tpl,2)} us, est: {tpl*51854708}") 

# Reading EDDB csv

In [ ]:

pgsql_params = dict(
    dsn=os.getenv("PGSQL_URL"),
    server_settings={'search_path': "eddb"}
)
pgpool = await asyncpg.create_pool(**pgsql_params)

In [ ]:
r = re.compile(r'''
    \s*                # Any whitespace.
    (                  # Start capturing here.
      [^,"']*?         # Either a series of non-comma non-quote characters.
      |                # OR
      "(?:             # A double-quote followed by a string of characters...
          [^"\\]|\\.   # That are either non-quotes or escaped...
       )*              # ...repeated any number of times.
      "                # Followed by a closing double-quote.
      |                # OR
      '(?:[^'\\]|\\.)*'# Same as above, for single quotes.
    )                  # Done capturing.
    \s*                # Allow arbitrary space before the comma.
    (?:,|$)            # Followed by a comma or the end of a string.
    ''', re.VERBOSE)




#filename='data/systems_recently-20220406.csv'
assert False
filename='data/systems.csv'
filesize=Path(filename).stat().st_size
chunksize = 32 * 1024 * 1024
est_count = int(filesize/chunksize) + 1
est_systems = 51854708
print(f"Reading {filename}, {int(filesize/(1024*1024))} Mb in approx {est_count} chunks")
count = 0
system_count = 0
columns = slice(2,6)

start = time.process_time()
with open(filename, "rt") as csvfile:
    headers = csvfile.readline()
    print(headers)
    while True:
        count += 1
        chunk = csvfile.readlines(chunksize)
        if chunk:
            data = []
            for line in chunk:
                name, *coordinates = r.findall(line)[columns]
                data.append(
                    [name.strip('"')] + 
                    [float(c) for c in coordinates])
            qr = await pgpool.copy_records_to_table("systems", records=data)
            system_count += len(data)
            print(f"Sytems: {system_count}\t{round(100*system_count/est_systems,2)}%,\t{int(system_count / (time.process_time() - start))} /s, {round(((est_systems - system_count) * (time.process_time() - start)/system_count),1)} remaining")
            continue
        print(f"Empy chunk -> Done! Imported {system_count} systems in {round(time.process_time() - start,1)} seconds")
        break

            

tpl = (time.process_time() - start)/system_count
print(f"{ (time.process_time() - start)} seconds {system_count} systems, per system {round(1000000*tpl,2)} us, est: {tpl*51854708}") 

## Check result

In [ ]:
print(await pgpool.fetchrow("SELECT * FROM systems WHERE name = $1", "Sol"))

In [ ]:
print(await pgpool.fetchrow("SELECT * FROM systems WHERE name = $1", "Deciat"))

In [ ]:
print(await pgpool.fetchrow("SELECT min(x), max(x) FROM systems"))

In [ ]:
print(await pgpool.fetchrow("SELECT min(y), max(y) FROM systems"))

In [ ]:
print(await pgpool.fetchrow("SELECT min(z), max(z) FROM systems"))

# Dividing the Galaxy

In [ ]:
# get the extend of the explored galaxy
qx = await pgpool.fetchrow("SELECT min(c20x), max(c20x) from systems")
qy = await pgpool.fetchrow("SELECT min(c20y), max(c20y) from systems")
qz = await pgpool.fetchrow("SELECT min(c20z), max(c20z) from systems")
print([(R.get("min"), R.get("max"), R.get("max")-R.get("min")) for R in [qx, qy, qz]])

In [ ]:
# get the extend of the explored galaxy
qx = await pgpool.fetchrow("SELECT min(x), max(x) from systems")
qy = await pgpool.fetchrow("SELECT min(y), max(y) from systems")
qz = await pgpool.fetchrow("SELECT min(z), max(z) from systems")
print([(R.get("min"), R.get("max"), R.get("max")-R.get("min")) for R in [qx, qy, qz]])

In [ ]:
rx = 

In [ ]:
known_extend = [(-42213.8125, 40503.8125, 82717.625), (-29359.8125, 39518.34375, 68878.15625), (-23405.0, 65630.15625, 89035.15625)]

In [ ]:
counts = [(int(math.floor(v[2]/200))) for v in known_extend]
print(counts)

In [ ]:
counts[0] * counts[1] * counts[2]

In [ ]:
pow(len(range(-40000, 40000, 200)),3)/pow(100,3)

In [ ]:
Xr, *YZ = [(int(200*math.floor(R.get("min")/200)) , int(200*math.floor(R.get("max")/200))) for R in [qx, qy, qz]]

In [ ]:
len(range(*Xr, 200))

In [ ]:
Yr, *Z = YZ

In [ ]:
len(range(*Yr, 200))

In [ ]:
Zr, *T = Z

In [ ]:
len(range(*Zr, 200))

In [ ]:
data = []
count = 0
start = time.process_time()

for cy in range(-3000, 3000, 200): # n=30
    count += 1
    qr = await pgpool.copy_records_to_table("volumes", records=[(cx,cy,cz)  for cx in range(-30000, 30000, 200) for cz in range(-24000, 66000,200)])
    print(f"{count}/{30}\t{round(100*count/30,2)}%, {round(((30 - count) * (time.process_time() - start)/count),1)} remaining")



In [ ]:
cy=0

In [ ]:
[(cx,cy,cz)  for cx in range(-30000, 30000, 2000) for cz in range(-24000, 66000,2000)]

In [ ]:
len(range(-24000, 66000, 200)) * len(range(-30000, 30000, 200)) * len(range(-3000, 3000, 200)) / 1000

In [ ]:
len(range(-30000, 30000, 200))